In [1]:
print("hello")

hello


In [1]:
import cv2
import numpy as np

In [ ]:
def mosaic(img, mosaic_rate):
    w = img.shape[1]
    h = img.shape[0]
    img = cv2.resize(img, (int(w * mosaic_rate), int(h * mosaic_rate)))
    img = cv2.resize(img, (w, h), interpolation = cv2.INTER_NEAREST)
    return img

cascade_file= cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
clas = cv2.CascadeClassifier(cascade_file)
cap = cv2.VideoCapture("output.mp4")

#動画を保存
fmt = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
fps = 20.0
size = (640, 360)
writer = cv2.VideoWriter('./test.mp4', fmt, fps, size)

while (cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        break

    #フレームのサイズを保存するサイズに合わせる
    frame=cv2.resize(frame, size)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_list = clas.detectMultiScale(gray, scaleFactor = 1.1, minNeighbors = 3, minSize=(30,30))

    for x, y, w, h in face_list:
        frame[y : y+h, x : x+w] = mosaic(frame[y : y + h, x : x + w], 0.05)

    #１フレームずつ画像を書き込む
    writer.write(frame)


    cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

writer.release()
cap.release()
cv2.destroyAllWindows()

YuNetのモデルファイルをダウンロード
(https://github.com/opencv/opencv_zoo/blob/main/models/face_detection_yunet/)

In [14]:
# 動画ファイルを開く
cap = cv2.VideoCapture("output2.mp4")

# 動画ファイルの各種プロパティーを取得
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # フレームの幅
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # フレームの高さ
fps = float(cap.get(cv2.CAP_PROP_FPS))  # FPS

print(f"フレームの幅: {frame_width}")
print(f"フレームの高さ: {frame_height}")
print(f"FPS: {fps}")

# YuNet顔検出モデルの読み込み
#yunet = cv2.FaceDetectorYN.create("face_detection_yunet_2023mar.onnx",
#                                  "", (frame_width, frame_height))


yunet = cv2.FaceDetectorYN.create(model="face_detection_yunet_2023mar_int8bq.onnx",
                                  config="",
                                  input_size=(frame_width, frame_height))

# VideoWriter を作成する。
output_file = "yu_net_output.mp4"  # 保存する動画ファイル名
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 動画のコーデックを指定
out = cv2.VideoWriter(output_file, fourcc, fps, (frame_width, frame_height), True)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 顔検出
    faces = yunet.detect(frame)

    # 顔の輪郭と目、鼻、口の位置を描画
    if faces[1] is not None:
        for idx, face in enumerate(faces[1]):
            coords = face[:-1].astype(np.int32)
            cv2.rectangle(frame, (coords[0], coords[1]), (coords[0]+coords[2], coords[1]+coords[3]), (0, 0, 255), 2)
            cv2.circle(frame, (coords[4], coords[5]), 2, (0, 255, 0), 2)
            cv2.circle(frame, (coords[6], coords[7]), 2, (0, 255, 0), 2)
            cv2.circle(frame, (coords[8], coords[9]), 2, (0, 255, 0), 2)
            cv2.circle(frame, (coords[10], coords[11]), 2, (0, 255, 0), 2)
            cv2.circle(frame, (coords[12], coords[13]), 2, (0, 255, 0), 2)

    # 動画にフレームを書き込む
    out.write(frame)

# リソースを開放
cap.release()
out.release()



フレームの幅: 1920
フレームの高さ: 1080
FPS: 59.94005994005994


memo
- 映像の画質が良くない
- 顔認証の制度が良くない
- 映像が反転している
- 1分の動画に3分処理時間がかかる（量子化を試したい）

ここで高速化を試みる

yolov8n-face.ptを以下からDL
https://github.com/akanametov/yolo-face?tab=readme-ov-file

In [ ]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO

# M3 MacのGPUを使用
device = torch.device("mps")

# YOLOv8の顔検出モデル（事前に適切な顔検出モデルをダウンロード）
model = YOLO("yolov8n-face.pt").to(device)  # 顔検出用のYOLOモデル

# 入出力ファイル設定
input_video = "input.mp4"
output_video = "output.mp4"

# 動画の読み込み
cap = cv2.VideoCapture(input_video)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 出力動画の設定
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

def apply_mosaic(image, x1, y1, x2, y2, mosaic_scale=0.1):
    """指定した座標の範囲にモザイクを適用"""
    face = image[y1:y2, x1:x2]
    h, w = face.shape[:2]

    # 縮小して拡大することでモザイク効果を作成
    small = cv2.resize(face, (max(1, int(w * mosaic_scale)), max(1, int(h * mosaic_scale))), interpolation=cv2.INTER_LINEAR)
    mosaic = cv2.resize(small, (w, h), interpolation=cv2.INTER_NEAREST)

    image[y1:y2, x1:x2] = mosaic
    return image

# フレーム処理
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 顔検出
    results = model(frame)

    # 各検出結果に対して処理
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # 顔の座標
            frame = apply_mosaic(frame, x1, y1, x2, y2, mosaic_scale=0.05)

    # 出力動画に書き込み
    out.write(frame)

# 後処理
cap.release()
out.release()
print("動画処理が完了しました。output.mp4 に保存されました。")

yolov8n-face.ptを以下からDL
https://github.com/akanametov/yolo-face?tab=readme-ov-file